## Import Dependencies 

In [71]:
import mediapipe as mp 
import cv2
import csv 
import os
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import RidgeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle

### Split Data into Training and Testing Sets

In [72]:
df = pd.read_csv('fer13.csv')

In [4]:
X = df.drop('class', axis=1)
y = df['class'] # target value

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

### Fit and Evaluate Different Models using pipelines

In [10]:
pipelines = {
    'ridge': make_pipeline(StandardScaler(), RidgeClassifier()),
    'randomForest':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'svm':make_pipeline(StandardScaler(), SVC()),
}

In [11]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train.values, y_train)
    fit_models[algo] = model

In [70]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test.values)
    print(algo, accuracy_score(y_test, yhat))

ridge 0.660548183058415
randomForest 0.5540121542850056
svm 0.5847699367481086


### Save and Load The Model with The Best Accuracy

In [15]:
with open('emotion_detection_rg1.pkl', 'wb') as f:
    pickle.dump(fit_models['ridge'], f)

In [16]:
with open('emotion_detection_rg1.pkl', 'rb') as f:
    model = pickle.load(f)

### Real-time Emotion Recognition

In [77]:
# initializing the landmarks drawing utilites 
mp_drawing = mp.solutions.drawing_utils
# mediapipe landmark detection models
mp_holistic = mp.solutions.holistic

capture = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  # looping through each frame recived from the webcam
  while (capture.isOpened()):
    ret, frame = capture.read()
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # convert image color format from BGR to RGB 
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    # pass the image to medipipe for detection 
    results = holistic.process(image)

    #convert the image color format back to BGR for rendering 
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

  # Draw face laandmarks 
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                              )
    # Draw right hand laandmarks 
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                              )
    # Draw left hand laandmarks 
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                              )
    # Draw pose laandmarks 
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                              )

    # Export Coordinates
    try:
        # Extract pose landmarks
        pose = results.pose_landmarks.landmark
        pose_row= list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]for landmark in pose]).flatten())

        # Extract face landmarks
        face = results.face_landmarks.landmark
        face_row= list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility]for landmark in face]).flatten())

        # Concate rows
        row = pose_row + face_row


        # Make Detections
        X = pd.DataFrame([row])
        emotion_class = model.predict(X.values)[0]
#         emotion_detection_prob = model.predict_proba(x.values)[0]

#          Grab ear coords
        coords = tuple(np.multiply(
                        np.array(
                            (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                            results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)
                        ), [640,480]).astype(int))
        
        cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(emotion_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
        cv2.putText(image, emotion_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
    
    except:
        pass
    

    cv2.imshow('Raw webcam Feed', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

capture.release()
cv2.destroyAllWindows()